# AutoEncoder

auto encoder 主要用于降维 dimension reduction. 同时也是unsupervised learning的主要使用工具.

主要逻辑就是重构输入reconstruct. 输入层和输出曾是一样大小的. 中间会有一个neck. 可以升维也可以降维, 一般都是降维. 这样我们就包含了语义相关性. neck的前半部分被称作encoder, 后半部分称为decoder.

![](./assets/27.png)

我们主要的对比就是输入和输出的误差最小. 这里比较的是每一个点的误差. 也就是输出出来的应该是一个特征类似的图谱. 所以可以使用MSE. 当然, 如果输入是一个二进制类型的输入(每一个pixel都是二进制的), 那么我们也可以使用交叉熵计算loss. 可以看出中间的一部分就是整个网络的精髓. 我们只需要创建一个晓得神经网络学习这个精髓. 

![](./assets/28.png)

由于autoencoder也是一个神经网络, 所以可以使用神经网络的各种特性, 比如dropout.

**Adversarial Autoencoder 对抗式自编码器**

相比于原始的autoencoder, 对抗式自编码器根据对抗网络创建了鉴别器. 在自编码生成器生成数据的时候, 我们训练的特征或者说属性对于每一个训练集都应该存在的, 这种属性的分布应该成一种正态分布. 这个时候我们使用一个鉴别器来查看我们生成的中间neck部分的结果是不是是一个正态分布, 或者我们自定义一种分布. 如果是的话, 那么我们就输出一个1或者0. 这样我们也能处理一些数据不均匀的问题.

![](./assets/29.png)

对于一个autoencoder, 我们网络中理想就是我们的得到的中间相的"变化程度"越接近输入的"变化程度"(就是相关性性很大). 因此就有了下面的公式, z就是我们的neck的结果, 我们希望给定$x_i$的时候会出现z的结果越大越好, 这样第一项也就是loss就趋近于零(q就是encoder网络的输出分布). 第二项KL曲线就是正态分布的重叠, 范围是0-inf. 如果两个分布相同的时候, KL就会趋近于0. 反之就会越来越大, 最大到正无穷. 因此我们希望如果使用在z出现的时候, x的分布和z的原始分布尽可能的相同, 否则的话KL函数就会越来越大, 这一项也就是误差.

总体来说, 下面的公式就是一个loss function, 第一项是来对比当输入x的时候同时得到z的结果概率, 这个概率进行log的结果应该是一个负值. 概率越大越趋近于0, 概率越小越趋近于负无穷. 第二项则是比较x和neck的分布, 如果分布差距过大, 那么第二项也会变大. 总体的error也就是一个正值.

![](./assets/30.png)

KL的计算公式如下, 结果就是两个分布的方差和均值进行计算.

![](./assets/31.png)

需要注意的是, 当我们在计算分布的时候我们没有办法得到所有的x和h, 因此我们得到的其实是一个抽样的结果, 最终得到一个$\sigma$和一个均值$\mu$. 但是这就出现了一个问题, 我们得到的这个loss function是没有办法反向传播或者微分的. 因此我们可以构建一个函数:

$$z=\mu+\sigma\cdot\epsilon$$

这个函数有一个好处, 均值和bias很像, 而且同时包含了所有的信息. 这里的$epsilon$就是正态分布

In [21]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets


In [22]:
# 读取数据集
mnist_train = datasets.MNIST(
    "../data",
    train=True,
    transform=transforms.Compose([transforms.ToTensor()]),
    download=True,
)
mnist_train = DataLoader(mnist_train, batch_size=32, shuffle=True)

mnist_test = datasets.MNIST(
    "../data",
    train=False,
    transform=transforms.Compose([transforms.ToTensor()]),
    download=True,
)
mnist_test = DataLoader(mnist_test, batch_size=32, shuffle=True)

x, _ = next(iter(mnist_train))
print("x:", x.shape)


x: torch.Size([32, 1, 28, 28])


In [23]:
# 创建autoencoder
# 输入的是一个28x28的图片
class MyAutoEncoder(nn.Module):
    def __init__(self) -> None:
        super(MyAutoEncoder, self).__init__()
        # [b, 784]
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU(),
        )
        # [b, 20]
        self.decoder = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid(),  # 将每个像素点压缩到0-1的区间, 使用sigmoid函数比较好
        )

    def forward(self, x):
        batchsz = x.size(0)
        x = x.view(batchsz, 784)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(batchsz, 1, 28, 28)
        return x


In [24]:
device = torch.device("mps")

model = MyAutoEncoder().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criteon = nn.MSELoss()
print(model)


MyAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=20, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
    (5): Sigmoid()
  )
)


In [26]:
# 开始训练
import visdom

viz = visdom.Visdom()


for epoch in range(10):
    for batchidx, (x, _) in enumerate(mnist_train):
        model.train()
        # [b, 1, 28, 28]
        x = x.to(device)
        x_hat = model(x)

        # 这里计算的就是整个图片的loss, 这里直接和原图比较
        loss = criteon(x_hat, x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 直接读取
    print("Epoch : {}, loss: {}".format(epoch, loss.item()))

    model.eval()
    x, _ = next(iter(mnist_test))
    x = x.to(device)
    with torch.no_grad():
        x_hat = model(x)
    viz.images(x, nrow=8, win="x", opts=dict(title="x"))
    viz.images(x_hat, nrow=8, win="x_hat", opts=dict(title="x_hat"))


Setting up a new session...


Epoch : 0, loss: 0.008594811893999577
Epoch : 1, loss: 0.00954330712556839
Epoch : 2, loss: 0.011033574119210243
Epoch : 3, loss: 0.008367041125893593
Epoch : 4, loss: 0.008513187989592552
Epoch : 5, loss: 0.009143290109932423
Epoch : 6, loss: 0.011294644325971603
Epoch : 7, loss: 0.008288084529340267
Epoch : 8, loss: 0.009100698865950108
Epoch : 9, loss: 0.009629801847040653


经过了多次的迭代和epochs我们得到的结果 

![](./assets/32.png)

### VAE

In [27]:
import numpy as np


In [34]:
# 相比于AE, VAE在进入neck之前, 使用了一次抽样的mean variance进行运算
class MyVAE(nn.Module):
    def __init__(self) -> None:
        super(MyVAE, self).__init__()
        # [b, 784]
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU(),
        )
        # 由于我们使用的是mean,variance, 所以维度减半
        # [b, 10]
        self.decoder = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid(),  # 将每个像素点压缩到0-1的区间, 使用sigmoid函数比较好
        )

    def forward(self, x):
        batchsz = x.size(0)
        x = x.view(batchsz, 784)
        # [b, 20] 包含mean和variance
        h = self.encoder(x)
        # 我们将所有的h按x轴拆分为两组
        # [b, 20] => [b, 10], [b, 10]
        mu, sigma = h.chunk(2, dim=1)
        # reparametrize trick
        # torch.randn_like() 方法返回一个具有与给定张量相同形状的随机张量，
        # 其元素按照标准正态分布（均值为0，标准差为1）进行采样。
        h = mu + sigma * torch.randn_like(sigma)
        x = self.decoder(h)
        x = x.view(batchsz, 1, 28, 28)

        # 计算KL divergence公式, 网上有, 上面也写了就是log(sigma_1/sigma_2)+....+-1/2那个
        # 注意, 这里拆分了上面的公式, 因为我们用的这个randn_like使用的是正态分布, 所以就是$N(0,1)$. 然后拆解公式得到了这个公式
        kld = (
            0.5
            * torch.sum(
                torch.pow(mu, 2)
                + torch.pow(sigma, 2)
                - torch.log(
                    1e-8 + torch.pow(sigma, 2)
                )  # 这里加了一个常数项1e-8主要是为了正则化, 防止计算log太小
                - 1,
            )
            / (batchsz * 28 * 28)
        )

        return x, kld


In [35]:
model = MyVAE().to(device)
criteon = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
print(model)


MyVAE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=20, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
    (5): Sigmoid()
  )
)


In [36]:
for epoch in range(10):
    for batchidx, (x, _) in enumerate(mnist_train):
        model.train()
        # [b, 1, 28, 28]
        x = x.to(device)
        x_hat, kld = model(x)

        # 这里计算的就是整个图片的loss, 这里直接和原图比较
        loss = criteon(x_hat, x)

        if kld is not None:
            elbo = -loss - 1.0 * kld
            loss = -elbo

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 直接读取
    print("Epoch : {}, loss: {}, kld: {}".format(epoch, loss.item(), kld.item()))

    model.eval()
    x, _ = next(iter(mnist_test))
    x = x.to(device)
    with torch.no_grad():
        x_hat, kld = model(x)
    viz.images(x, nrow=8, win="x", opts=dict(title="x"))
    viz.images(x_hat, nrow=8, win="x_hat", opts=dict(title="x_hat"))


Epoch : 0, loss: 0.063406802713871, kld: 0.015677781775593758
Epoch : 1, loss: 0.050572168081998825, kld: 0.008988361805677414
Epoch : 2, loss: 0.04980769753456116, kld: 0.009108103811740875
Epoch : 3, loss: 0.049099698662757874, kld: 0.009766747243702412
Epoch : 4, loss: 0.04694897308945656, kld: 0.009697605855762959
Epoch : 5, loss: 0.04785001650452614, kld: 0.010103937238454819
Epoch : 6, loss: 0.04577450826764107, kld: 0.00970194861292839
Epoch : 7, loss: 0.05065711960196495, kld: 0.009989737533032894
Epoch : 8, loss: 0.04721555858850479, kld: 0.01039033941924572
Epoch : 9, loss: 0.048142530024051666, kld: 0.010529637336730957


输出结果看上去不如AE

![](./assets/33.png)